In [2]:
import warnings
warnings.filterwarnings(action='ignore') 
import time
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# 2015 ~ 2020 서비스 업종별 매출 및 폐업률 데이터 read
for i in range(2015,2021):
    globals()["_{}y_hat_revenue".format(i)] = pd.read_csv(r'/content/drive/My Drive/OSSP_상권분석_proj/예상매출&폐업률/서울시 우리마을가게 상권분석서비스(상권-추정매출)_'+str(i)+'_new.csv',encoding='cp949')
    globals()["_{}y_fail_rate".format(i)] = pd.read_csv(r'/content/drive/My Drive/OSSP_상권분석_proj/예상매출&폐업률/서울시 우리마을가게 상권분석서비스(상권-점포)_'+str(i)+'_new.csv',encoding='cp949')


In [19]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

In [4]:
_2015y_hat_revenue

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액
0,0,2015,1,D,발달상권,1001018,포이사거리_2,CS300021,문구,3390014653
1,1,2015,1,A,골목상권,1000481,성미산로11길,CS300011,일반의류,477810637
2,2,2015,1,A,골목상권,1000440,세검정로1길,CS200008,한의원,29303647
3,3,2015,1,R,전통시장,1001458,인헌시장,CS100005,제과점,312469941
4,4,2015,1,A,골목상권,1000756,사당로8길,CS200016,당구장,38474191
...,...,...,...,...,...,...,...,...,...,...
164372,164372,2015,4,A,골목상권,1000295,장월로1길,CS200017,골프연습장,2441860
164373,164373,2015,4,A,골목상권,1000488,신촌로12길,CS200041,사진관,1151515
164374,164374,2015,4,A,골목상권,1000207,제기로5길,CS300002,편의점,479803060
164375,164375,2015,4,A,골목상권,1000607,화곡로59길,CS200031,세탁소,48549293


In [5]:
_2020y_hat_revenue

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액
0,0,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,6415600
1,1,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,1853749205
2,2,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,10400909742
3,3,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,807257363
4,4,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300027,섬유제품,1410497003
...,...,...,...,...,...,...,...,...,...,...
66506,66506,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129
66507,66507,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306
66508,66508,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866
66509,66509,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161


In [6]:
_2015y_fail_rate

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,폐업_률
0,0,2015,1,R,전통시장,1001490,길동복조리시장,CS300011,일반의류,3
1,1,2015,1,D,발달상권,1001046,서울 관악구 신림역_4,CS300001,슈퍼마켓,0
2,2,2015,1,A,골목상권,1000609,개봉로11길,CS100001,한식음식점,0
3,3,2015,1,A,골목상권,1000910,도산대로81길,CS100004,양식음식점,33
4,4,2015,1,A,골목상권,1000862,방배로18길,CS100002,중식음식점,0
...,...,...,...,...,...,...,...,...,...,...
347400,347400,2015,4,D,발달상권,1001229,서울 종로구 종로5가역_4,CS300014,신발,0
347401,347401,2015,4,U,관광특구,1001494,종로·청계 관광특구,CS200011,변리사사무소,0
347402,347402,2015,4,A,골목상권,1000009,삼청로5길,CS300027,섬유제품,0
347403,347403,2015,4,A,골목상권,1000012,옥인길,CS200037,노래방,0


In [7]:
_2020y_fail_rate

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,폐업_률
0,0,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,0
1,1,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300042,주유소,0
2,2,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300041,예술품,0
3,3,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300040,재생용품 판매점,0
4,4,2020,2,U,관광특구,1001496,강남 마이스 관광특구,CS300038,자동차부품,0
...,...,...,...,...,...,...,...,...,...,...
184550,184550,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,0
184551,184551,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,0
184552,184552,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,0
184553,184553,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,0


In [8]:
# 2015 ~ 2020 추정매출 수직 통합

_total_hat_revenue = pd.DataFrame()
_total_fail_rate = pd.DataFrame()


for i in range ( 2015, 2021 ):
    _total_hat_revenue = _total_hat_revenue.append( globals()["_{}y_hat_revenue".format(i)], ignore_index = True )
    
for i in range ( 2015, 2021 ):
    _total_fail_rate = _total_fail_rate.append( globals()["_{}y_fail_rate".format(i)], ignore_index = True )

In [9]:
_total_hat_revenue

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액
0,0,2015,1,D,발달상권,1001018,포이사거리_2,CS300021,문구,3390014653
1,1,2015,1,A,골목상권,1000481,성미산로11길,CS300011,일반의류,477810637
2,2,2015,1,A,골목상권,1000440,세검정로1길,CS200008,한의원,29303647
3,3,2015,1,R,전통시장,1001458,인헌시장,CS100005,제과점,312469941
4,4,2015,1,A,골목상권,1000756,사당로8길,CS200016,당구장,38474191
...,...,...,...,...,...,...,...,...,...,...
862311,66506,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129
862312,66507,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306
862313,66508,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866
862314,66509,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161


In [10]:
_total_fail_rate

,Unnamed: 0,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,폐업_률
0,0,2015,1,R,전통시장,1001490,길동복조리시장,CS300011,일반의류,3
1,1,2015,1,D,발달상권,1001046,서울 관악구 신림역_4,CS300001,슈퍼마켓,0
2,2,2015,1,A,골목상권,1000609,개봉로11길,CS100001,한식음식점,0
3,3,2015,1,A,골목상권,1000910,도산대로81길,CS100004,양식음식점,33
4,4,2015,1,A,골목상권,1000862,방배로18길,CS100002,중식음식점,0
...,...,...,...,...,...,...,...,...,...,...
1969588,184550,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,0
1969589,184551,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,0
1969590,184552,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,0
1969591,184553,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,0


In [11]:
_total_hat_revenue = _total_hat_revenue.drop(['Unnamed: 0','상권_구분_코드','상권_구분_코드_명','상권_코드_명','서비스_업종_코드_명'], axis=1)
_total_hat_revenue

,기준_년_코드,기준_분기_코드,상권_코드,서비스_업종_코드,당월_매출_금액
0,2015,1,1001018,CS300021,3390014653
1,2015,1,1000481,CS300011,477810637
2,2015,1,1000440,CS200008,29303647
3,2015,1,1001458,CS100005,312469941
4,2015,1,1000756,CS200016,38474191
...,...,...,...,...,...
862311,2020,1,1000001,CS100005,257814129
862312,2020,1,1000001,CS100004,156728306
862313,2020,1,1000001,CS100003,55484866
862314,2020,1,1000001,CS100002,25767161


In [12]:
_total_fail_rate = _total_fail_rate.drop(['Unnamed: 0','상권_구분_코드','상권_구분_코드_명','상권_코드_명','서비스_업종_코드_명'], axis=1)
_total_fail_rate


,기준_년_코드,기준_분기_코드,상권_코드,서비스_업종_코드,폐업_률
0,2015,1,1001490,CS300011,3
1,2015,1,1001046,CS300001,0
2,2015,1,1000609,CS100001,0
3,2015,1,1000910,CS100004,33
4,2015,1,1000862,CS100002,0
...,...,...,...,...,...
1969588,2020,1,1000001,CS100005,0
1969589,2020,1,1000001,CS100004,0
1969590,2020,1,1000001,CS100003,0
1969591,2020,1,1000001,CS100002,0


In [13]:
columns = _total_fail_rate.columns
key_columns = list(columns)
key_columns.remove('폐업_률')
print(key_columns[:5])

['기준_년_코드', '기준_분기_코드', '상권_코드', '서비스_업종_코드']


In [14]:
_total = pd.merge(_total_hat_revenue,_total_fail_rate, how='outer',on=key_columns)

In [15]:
_total

,기준_년_코드,기준_분기_코드,상권_코드,서비스_업종_코드,당월_매출_금액,폐업_률
0,2015,1,1001018,CS300021,3390014653,0
1,2015,1,1000481,CS300011,477810637,14
2,2015,1,1000440,CS200008,29303647,0
3,2015,1,1001458,CS100005,312469941,0
4,2015,1,1000756,CS200016,38474191,100
...,...,...,...,...,...,...
1977759,2020,1,1000001,CS200005,nan,0
1977760,2020,1,1000001,CS200003,nan,0
1977761,2020,1,1000001,CS200002,nan,0
1977762,2020,1,1000001,CS100009,nan,0


In [16]:
is_NaN = _total.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = _total[row_has_NaN]
rows_with_NaN

,기준_년_코드,기준_분기_코드,상권_코드,서비스_업종_코드,당월_매출_금액,폐업_률
17,2015,1,1001380,CS100006,19481235,nan
29,2015,1,1000439,CS100004,7119785,nan
298,2015,1,1000904,CS300025,14802156,nan
444,2015,1,1000431,CS200008,48078881,nan
477,2015,1,1000204,CS200005,19004551,nan
...,...,...,...,...,...,...
1977759,2020,1,1000001,CS200005,nan,0
1977760,2020,1,1000001,CS200003,nan,0
1977761,2020,1,1000001,CS200002,nan,0
1977762,2020,1,1000001,CS100009,nan,0


In [22]:
_total['서비스_업종_코드'].isnull().any()

False

In [23]:
# onehot = OneHotEncoder()
# x = onehot.fit_transform(_total['상권_코드'].values.reshape(-1,1)).toarray()

# onehot_df = pd.DataFrame(x, columns = ["상권코드_"+str(int(i)) for i in range(x.shape[1])])
# _total_o_hot = pd.concat([_total, onehot_df], axis=1)
# _total_o_hot

In [26]:
from sklearn.preprocessing import LabelEncoder

items= _total['서비스_업종_코드'].unique()

encoder = LabelEncoder()
encoder.fit(items)

labels = encoder.transform(items)

In [27]:
print('인코딩 변환값:',labels)

인코딩 변환값: [77 67 17  4 25  8 78 21 85  6 37 16 15 63  3  5 11 70 57 46 58 99 62 50
 14  1 38 90 66 47 84 33 76  0 74 40 83 65 72 71  7  2 89 94 59 91 88 10
  9 39 92 53 80 12 28 75 60 64 26 43 69 81 35 34 73 98 24 87 82 45 41 42
 23 68 19 97 27 32 52 93 29 54 79 22 55 95 56 13 96 30 44 48 61 20 36 18
 49 51 86 31]


In [29]:
_total['서비스_업종_코드'] = encoder.transform(_total['서비스_업종_코드'])

ValueError: ignored

In [30]:
_total

,기준_년_코드,기준_분기_코드,상권_코드,서비스_업종_코드,당월_매출_금액,폐업_률
0,2015,1,1001018,77,3390014653,0
1,2015,1,1000481,67,477810637,14
2,2015,1,1000440,17,29303647,0
3,2015,1,1001458,4,312469941,0
4,2015,1,1000756,25,38474191,100
...,...,...,...,...,...,...
1977759,2020,1,1000001,14,nan,0
1977760,2020,1,1000001,12,nan,0
1977761,2020,1,1000001,11,nan,0
1977762,2020,1,1000001,8,nan,0


In [ ]:
start = time.time()  # 시작 시간 저장

imputer = KNNImputer(n_neighbors = 15000 )

_total_knn = imputer.fit_transform(_total)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
_total_knn.to_csv(r'/content/drive/My Drive/OSSP_상권분석_proj/예상매출&폐업률/_total_knn.to_csv.csv',encoding='cp949')
